In [2]:
import time, sys, os, subprocess, serial, json
import pandas as pd
import numpy as np
import plotly.express as px

In [3]:
def read_data_to_csv(file_path, filename):
    for k in range(5):
        if not os.path.exists(file_path):
            os.makedirs(file_path)
        ser = serial.Serial('COM4', 115200, timeout=1)
        linesToRead = 1000
        data = []
        for i in range(0, linesToRead):
            try:
                line = ser.readline().decode('utf-8')
                data.append(line)
            except:
                print("Error reading from serial port")
                continue
        ser.close()

        data_arr = []
        
        for line in data:
            try:
                data_arr.append(json.loads(line))
            except:
                continue
            #load all json data into a dataframe
            df = pd.json_normalize(data_arr)
            if not os.path.exists(file_path):
                os.makedirs(file_path)
            df.to_csv(os.path.join(file_path, filename + '_' + str(k) + '.csv'))

In [4]:
device = 'COM3'
baudrate = 115200
result_dir = 'negative_task_1'
project_dir = r'C:\Users\celes\OneDrive\Documents\PlatformIO\Projects\imu'
main_file = r'main.cpp'
bat_file = 'test.bat'
options = {
    'filter_settings': ['SET_FILTER_5HZ', 'SET_FILTER_10HZ', 'SET_FILTER_21HZ', 'SET_FILTER_44HZ', 'SET_FILTER_94HZ', 'SET_FILTER_184HZ', 'SET_FILTER_260HZ'],
    'accelerometer_range': ['SET_ACC_2G', 'SET_ACC_4G', 'SET_ACC_8G', 'SET_ACC_16G'],
    'gyroscope_range': ['SET_GYRO_250', 'SET_GYRO_500', 'SET_GYRO_1000', 'SET_GYRO_2000']
}

keywords = ['@filter_setting', '@accelerometer_setting', '@gyroscope_setting']
for filter_setting in options['filter_settings']:
    for accelerometer_range in options['accelerometer_range']:
        for gyroscope_range in options['gyroscope_range']:
            #open the file main.cpp and change %FILTER_SETTING% to filter_setting
            template = ""
            filedata = open(main_file, 'r').read()
            filedata = filedata.replace(keywords[0], filter_setting)
            filedata = filedata.replace(keywords[1], accelerometer_range)
            filedata = filedata.replace(keywords[2], gyroscope_range)
            # overwrite the file
            with open(project_dir + '\src\main.cpp', 'w') as file:
                file.write(filedata)
                file.close()
            p = subprocess.Popen(bat_file, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
            os.system('cls')
            lines = p.stdout.readlines()
            build_success = False
            for line in lines:
                if line.decode('utf-8').find('SUCCESS') != -1:
                    print(line.decode('utf-8'))
                    build_success = True
                    break
            if not build_success:
                print('Build failed')
                sys.exit()
            p.wait()
            #wait 5 seconds for the program to run
            time.sleep(5)
            #read the data from the serial port
            read_data_to_csv(result_dir + r'\data', filter_setting + '_' + accelerometer_range + '_' + gyroscope_range)


"Starting PlatformIO"

Hello World 

Processing d1_mini_lite (platform: espressif8266; board: d1_mini_lite; framework: arduino)

--------------------------------------------------------------------------------

Verbose mode can be enabled via `-v, --verbose` option

CONFIGURATION: https://docs.platformio.org/page/boards/espressif8266/d1_mini_lite.html

PLATFORM: Espressif 8266 (4.0.1) > WeMos D1 mini Lite

HARDWARE: ESP8266 80MHz, 80KB RAM, 1MB Flash

PACKAGES: 

 - framework-arduinoespressif8266 @ 3.30002.0 (3.0.2) 

 - tool-esptool @ 1.413.0 (4.13) 

 - tool-esptoolpy @ 1.30000.201119 (3.0.0) 

 - tool-mklittlefs @ 1.203.210628 (2.3) 

 - tool-mkspiffs @ 1.200.0 (2.0) 

 - toolchain-xtensa @ 2.100300.210717 (10.3.0)

LDF: Library Dependency Finder -> https://bit.ly/configure-pio-ldf

LDF Modes: Finder ~ chain, Compatibility ~ soft

Found 35 compatible libraries

Scanning dependencies...

Dependency Graph

|-- Wire @ 1.0

Building in release mode

Compiling .pio\build\d1_mini_lite\src